# Modify Nodes in an NDEx Network Based on a TSV File

In this notebook, you will modify a network from NDEx that shows the interactions between the NCI ITCR projects.

This version is missing some node attributes and needs corrections to the organization of their attributes.

The network will be updated based on a TSV file where each row has information about one of the ITCR projects.

The workflow:
 * Load the TSV file as a Pandas Dataframe
 * Create an NiceCX network from the network on NDEx
 * For each row in the Dataframe:
  * Identify the corresponding node (skip if there is no such node)
  * Add attributes to the node based on the following columns:
   * “Category”
   * “Tool URL”
   * “Code Repo”
   * “Project Info” 
  * Set the "Tool Description" attribute of the node to the value of the "Body" column
  * Set the node name to be the value of the "Title" column
  * Set the "label" attribute to the value of the "Tool Acronym" when available, otherwise use the value of the "Title" column.
 * Save the network to your account on NDEx

The final network should still have 36 nodes/81 edges but provide extra info and link out to external resource

First, import the ndex2 Python client, Pandas, and itertools

In [1]:
import ndex2
import pandas as pd
import itertools

Next, set a variable to hold the UUID of the network we want to modify

In [2]:
my_network_uuid = '04c0a7e8-af92-11e7-94d3-0ac135e8bacf'

Set variables to hold your account and password 

In [3]:
my_server = 'public.ndexbio.org'
my_account = "drh"
my_password = "drh"

Create a NiceCX network from the ITCR network..

In [24]:
print("downloading network and buiding NiceCX...")
itcr_network = ndex2.create_nice_cx_from_server(server=my_server, uuid=my_network_uuid)
#my_network = ndex2.create_nice_cx_from_server(server='public.ndexbio.org', uuid=my_network_uuid, account=my_account, password = my_password)
print("done")
print(itcr_network.get_summary())

downloading network and buiding NiceCX...
done
Name: ITCR Connectivity Map
Nodes: 36
Edges: 81
Node Attributes: 0
Edge Attributes: 241



Load the TSV to a Datframe

In [5]:
tsv_path = '../resources/itcr-conn-map-v2.0_nodes.txt'
node_data = pd.read_csv(tsv_path, sep="\t")
node_data

,Title,Tool Acronym,Body,Category,Tool URL,Code Repo,Project Info
0,3D Slicer,NaN,3D Slicer is the free open source software for...,Imaging,https://www.slicer.org,https://github.com/Slicer,https://projectreporter.nih.gov/project_info_d...
1,Allele-Specific Alternative mRNA processing (A...,ASARP,A software pipeline for prediction of allele-s...,omics,https://www.ibp.ucla.edu/research/xiao/Softwar...,https://github.com/cyruschan/ASARP,https://projectreporter.nih.gov/project_info_d...
2,Apache Clinical Text and Knowledge Extraction ...,cTAKES,The tool extracts deep phenotypic information ...,Clinical,http://ctakes.apache.org,http://ctakes.apache.org/downloads.cgi,https://projectreporter.nih.gov/project_info_d...
3,Bioconductor,NaN,Bioconductor provides tools for the analysis a...,Omics,https://www.bioconductor.org,http://bioconductor.org/developers/how-to/git/,https://projectreporter.nih.gov/project_info_d...
4,Cancer and Phenomics Toolkit (CaPTK),CaPTK,CaPTk is a tool that facilitates translation o...,Imaging,http://captk.projects.nitrc.org,NaN,https://projectreporter.nih.gov/project_info_d...
5,Cancer Slide Digital Archive (CDSA),CDSA,The CDSA is a web-based platform to support th...,Imaging,http://cancer.digitalslidearchive.net,NaN,https://projectreporter.nih.gov/project_info_d...
6,Cancer-Related Analysis of Variants Toolkit (C...,CRAVAT,CRAVAT is an easy to use web-based tool for an...,Omics,http://www.cravat.us/CRAVAT/,https://github.com/KarchinLab/CRAVAT,https://projectreporter.nih.gov/project_info_d...
7,Cistrome,NaN,Curated and processed human/mouse ChIP/DNase-s...,Omics,http://cistrome.org/Cistrome/Cistrome_Project....,NaN,https://projectreporter.nih.gov/project_info_d...
8,Clinical Interpretation of Variants in Cancer ...,CIViC,"CIViC is an open access, open source, communit...","Omics, Clinical",https://civic.genome.wustl.edu/home,https://github.com/griffithlab,https://projectreporter.nih.gov/project_info_d...
9,Cores Of Recurrent Events (CORE),CORE,CORE is a statistically supported computationa...,Omics,NaN,http://cran.us.r-project.org/web/packages/CORE/,https://projectreporter.nih.gov/project_info_d...


Annotate the nodes in the network according to the plan.


In [26]:
name_to_node_map = {}
for index, node in itcr_network.get_nodes():
    name_to_node_map[node.get_name()] = node
        
for index, row in node_data.iterrows():
    # Identify the corresponding node (skip if there is no such node)
    node = name_to_node_map.get(row['Title']) or name_to_node_map.get(row['Tool Acronym'])
    if node:
        
        # Add attributes to the node
        attributes = ["Category", "Tool URL", "Code Repo", "Project Info"]
        for att in attributes:
            itcr_network.set_node_attribute(node, att, row[att])
            
        # Set the "Tool Description" attribute of the node to the value of the "Body" column
        itcr_network.set_node_attribute(node, "Tool Description", row['Body'])
        
        # Set the node name to be the value of the "Title" column
        #node.set_name(row["Title"])
        
        # Set the "label" attribute to the value of the "Tool Acronym" when available, 
        # otherwise use the value of the "Title" column.
        acronym = row["Tool Acronym"]
        if not isinstance(acronym, str):
            acronym = None
        itcr_network.set_node_attribute(node, "label", acronym or row["Title"] or "no label")


Spot-check the network by printing out a sample of the nodes in the modified network

In [27]:
print(itcr_network.get_summary())

for id, node in itertools.islice(itcr_network.get_nodes(), 5):
    label = itcr_network.get_node_attribute(node, "label")
    tool = itcr_network.get_node_attribute(node, "Tool Description")
    print("%s: %s = %s" % (node.get_name(), label, tool))

Name: ITCR Connectivity Map
Nodes: 36
Edges: 81
Node Attributes: 174
Edge Attributes: 241

TIES: TIES = TIES is a service based software system for acquiring, deidentifying, and processing clinical text reports using natural language processing, and also for querying, sharing and using this data to foster tissue and image based research, within and between institutions.

TCPA: None = None
HIVE (BioExpress): None = None
TCIA: TCIA = TCIA is NCI’s repository for publicly shared cancer imaging data. TCIA collections include radiology and pathology images, clinical and clinical trial data, image derived annotations and quantitative features and a growing collection of related ‘omics data both from clinical and pre-clinical studies.

OmniSearch: OmniSearch = OmniSearch is a semantic search software based upon the OMIT ontology. While OmniSearch is by its nature extensible, its initial focus is in human cancer research.



Upload the network to your NDEx account as a new network.

If you started by loading an one of your networks (such as a clone of the example network), then you can *update* it instead, overwriting its content. 

In the commented line of below, the optional parameter *update_uuid* specifies the update behavior.

In [ ]:
upload_message = my_network.upload(my_server, my_account, my_password)
# upload_message = my_network.upload(my_server, my_account, my_password, update_uuid=my_network_uuid)
print(upload_message)